# Demonstration on Simulated fMRI Data   

Author: Xuanzhi CHEN <xuanzhichen.42@gmail.com>

License: MIT License

<!--
# ### DEVELOPMENT NOTES ####################################################
# *


# ### DEVELOPMENT PROGRESS (LEAST) #########################################
# * Doing on getting seeds. First, go to the experiment helper to write fun-
    ction run_generation_procedure(). Then, get_skeleton_score() and get_sk-
    eleton_from_pc() can be the standard module from utils.   08th.Dec, 2023
   

# ### TO-DO LIST (LEAST) ###################################################
# Required (Optional):
# TODO: experiment_helper.run_generation_procedure(), get_skeleton_score() 
        and get_skeleton_from_pc()
# Done:
# None
-->

To illustrate our experiments, we simply set repetitions equal to 10. 

Computational cost required to reproduce the corresponding experiment need to be approximately four to five day full-time computation by our roughly estimation.  

Machine: 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import time
import warnings

from experiment_helper import ExperimentHelper
from cadimulc.utils.extensive_modules import copy_and_rename

plt.style.use("seaborn-whitegrid")
warnings.filterwarnings("ignore")

## Experimental Setup

### Main Parameters

| Parameters       | Descriptions                                                                                                                                                                                     | 
|:-----------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| models           | The causal discovery algorithms selected as baseline approaches include: CAM-UV[cite], RESIT[cite], FCI[cite], PC[cite], compared with the novel method **Nonlinear-MLC** proposed in our paper. | 
| scenarios_index  | testing text                                                                                                                                                                                     |
| exp_repetitions  | testing text                                                                                                                                                                                        | 
| saving_data      | testing text                                                                                                                                                                              | 
| saving_data_path | testing text                                                                                                                                                                                          |
| saving_fig       | testing text                                                                                                                                                                                |
| saving_fig_path  | testing text                                                                                                                                                                                          |

In [7]:
models = ["Nonlinear-MLC", "CAM-UV", "RESIT", "FCI", "PC"]

fmri_dataset_id = 'sim2_dataset'
fmri_dataset_path = '../dataset_netsim/sim2.mat'

latent_var_dic = {
    'sim2_dataset' : ('X0', 'X5'),
    'sim3_dataset' : ('X0', 'X5', 'X10')
}
latent_var_set = latent_var_dic[fmri_dataset_id]

sample = 1000

# exp_repetitions = 100
exp_repetitions = 10

plot = True
plotting_title = "Performance on fMRI data of sim2"

saving_data = True
data_save_path = 'record_data/date_folder_name/group1'
saving_img = True
img_save_path = 'record_img/date_folder_name/group1'

### Convenience Functions

| Functions        | Descriptions                                                                                                                                                                                     | 
|:-----------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| display_exp_para           | testing text | 
| init_exp_helper  | testing text                                                                                                                                                                                     |
| get_seed_list  | testing text                                                                                                                                                                                        | 

In [8]:
def display_exp_para():
    print("######################### REVIEW EXPERIMENTAL SETUP #########################")
    print("------------------------- OPERATING PARAMETERS -------------------------")
    print("* fMRI-Dataset:              ", fmri_dataset_id)
    print("* Latent Variables Set:      ", latent_var_set)
    print("* Models:                    ", models)
    print("* Number of Repetitions:     ", exp_repetitions)
    
    identifier_end = '-------------------------'
    
    if plot:
        identifier_center = ' PLOTTING PARAMETERS '
        print(identifier_end + identifier_center + identifier_end)
        print("* Plotting Title:            ", plotting_title)
    
    if saving_data or saving_img:
        identifier_center = ' SAVE PARAMETERS '
        print(identifier_end + identifier_center + identifier_end)
        if saving_data:
            print("* Data Save Path:            ", data_save_path)
        if saving_img:
            print("* Image Save Path:           ", img_save_path)

In [ ]:
def init_exp_helper():
    """.
    Return a 'exp_helper' dictionary that ...
    """
    exp_helper = {}
    
    for model in models:
        exp_helper[model] = None
        
    return exp_helper

In [ ]:
def get_seed_list():
    """
    Write down some descriptions here.
    """
    pass

In [84]:
            
# Store result for preview.
presicion_y = {}
recall_y = {}
f1_score_y = {}
computational_time_y = {}
dict_list = [presicion_y, recall_y, f1_score_y, computational_time_y]

for dict_item in dict_list:
    for model in models:
        dict_item[model] = []

def initialize_exp_helper():
    exp_helper = {}
    for model in models:
        exp_helper[model] = None

    return exp_helper

def characterized(setting_value):
    """ Transform the setting value to string if it is about "ratio" type.
    """
    if type(setting_value) is float:
        temp = str(setting_value).split(".")
        setting_value = ""
        for char in temp:
            setting_value += char
    else:
        setting_value = str(setting_value)

    return setting_value

#### *temp init_random_seed_dict( )

In [1]:
def init_random_seed_dict():
    random_seed_dict = {
    0 : [],
    1 : [],
}

### Display Experimental Setup

In [6]:
display_exp_para()

######################### REVIEW EXPERIMENTAL SETUP #########################
------------------------- OPERATING PARAMETERS -------------------------
* fMRI-Dataset:               sim2_dataset
* Latent Variables Set:       ('X0', 'X5')
* Models:                     ['Nonlinear-MLC', 'CAM-UV', 'RESIT', 'FCI', 'PC']
* Number of Repetitions:      100
------------------------- PLOTTING PARAMETERS -------------------------
* Plotting Title:             Performance on fMRI data of sim2
------------------------- SAVE PARAMETERS -------------------------
* Data Save Path:             record_data/date_folder_name/group1
* Image Save Path:            record_img/date_folder_name/group1


## Markov and Faithfulness Assumptions

### Definition

Write down some descriptions here.

### Support of Hypothesis

We employ a heuristic approach to support our hypotheis.

The function is commonly used in the preprocessing stage to select datasets
    from which reasonable skeletons can be recovered. The higher the score
    an estimated skeleton achieves, the stronger the Markov and faithfulness
    assumptions approximately hold.

#### *temp: get_skeleton_score( )

In [ ]:
def get_skeleton_score(est_skeleton, true_skeleton, score_metric):
    """
    The function is commonly used in the preprocessing stage to select datasets
    from which reasonable skeletons can be recovered. The higher the score
    an estimated skeleton achieves, the stronger the Markov and faithfulness
    assumptions approximately hold.

    Parameters
    ----------
    est_skeleton : ndarray
    true_skeleton : ndarray
    score_metric : string

    Return
    ------
    score : float
        Precision, Recall, and F1-score
    """

    est_skeleton = convert_graph_type(origin=est_skeleton, target=nx.Graph)
    true_skeleton = convert_graph_type(origin=true_skeleton, target=nx.Graph)

    est_edges = list(est_skeleton.edges())
    true_edges = list(true_skeleton.edges())
    true_est_num = 0

    for est_edge in est_edges:
        if est_edge in true_edges:
            true_est_num += 1

    precision = 0
    recall = 0
    f1_score = 0

    if score_metric == 'Precision':
        score = precision
    elif score_metric == 'Recall':
        score = recall
    elif score_metric == 'F1-score':
        score = f1_score
    else:
        raise ValueError("test")

    return score

#### *temp get_skeleton_from_pc( )

In [ ]:
def get_skeleton_from_pc():
    pass

#### *temp get seed

In [ ]:
random_seed_dict = init_random_seed_dict()
experiment_helper = ExperimentHelper(
    fmri_dataset_id=fmri_dataset_id, 
    latent_var_set=latent_var_set,
)

for hidden_num in range(len(latent_var_set)):
    num_random_seeds = 0
    seed_candidate = 0
    while num_random_seeds < exp_repetitions:
        np.random.seed(seed_candidate)
        
        data, true_skeleton = experiment_helper.run_generation_procedure(
            hidden_num = hidden_num
        ).unpack()
        
        est_skeleton = get_skeleton_from_pc(data)
        score = get_skeleton_score(true_skeleton, est_skeleton)
        
        if score > 0.9:
            random_seed_dict[hidden_num].append(seed_candidate)
            
            num_random_seeds += 1
            
print(random_seed_dict)

## Conduct Experiments

### Running Process

In [ ]:
exp_helper = init_exp_helper() 
seed_list = get_seed_list()

In [ ]:
begin = time.perf_counter()

for model in models:
    
    exp_helper[model] = ExperimentHelper()
    
    for hidden_num in range(len(latent_var_set)):
        for seed in random_seed_dict[hidden_num]:
            
            exp_helper[model].run_generation_procedure()
            
            exp_helper[model].run_algorithm(model=model)
            
            exp_helper[model].run_evaluation_procedure()
        
        exp_helper[model].save_result()
            
begin = time.perf_counter()

In [ ]:
# time
show_exp_time()

In [ ]:
# preview
preview_exp_results()

### Data Processing and Result Plotting

## Reference